In [2]:
import os
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import date
import sys
from datetime import datetime , timedelta, date
import time
from datetime import date
from dateutil.relativedelta import relativedelta
from pathlib import Path
from glob import glob 
import shutil
import zipfile
import pandas as pd
import schemas

from loguru import logger

In [2]:
PATH_BASE = r"C:\Users\thiag\Documents\BASE_SUSEP"
os.makedirs(PATH_BASE,exist_ok=True)
time.sleep(1)
logger.add(os.path.join(PATH_BASE,'LOGS_EXTRACAO_SUSEP.log'),level='INFO')
logger.add(os.path.join(PATH_BASE,'LOGS_EXTRACAO_SUSEP_ERROS.log'),level='ERROR')


PATH_PASTA_ARQUIVOS = os.path.join(PATH_BASE,"Download_arquivo")
PATH_PASTA_ARQUIVOS_CSV = os.path.join(PATH_BASE,"Arquivos_CSV")
QDE_MAX_SEGUNDOS_DOWNLOAD = 600 




In [3]:
def criar_pasta(path):
    '''
    Objetivo da função: Apagar e recriar a pasta recebida no argumento Path

    '''

    try:
        shutil.rmtree(path,ignore_errors=True)
    except:
        ...

    os.makedirs(path,exist_ok=True)

In [ ]:
def buscar_arquivo_na_susep(path:Path ,qde_max_segundos_download:int) -> bool:
    '''
    Objetivo da função: Acessa o site da Susep e efetua o download da base ZIP
    Args:
        path (Path): Caminho onde iremos salvar o download do arquivo
        qde_max_segundos_download (int): Tempo máximo em segundos que o sistema irá esperar o download ser concluído, ao atingor o limite o processo será abortado  

    Returns:
        bool: Se o download for concluído com succeso retornará True caso contrário False 
        
    '''
    logger.info('Efetuando download do Zip')

    try:
        check_download = False
        qde_max_segundos = qde_max_segundos_download
        criar_pasta(PATH_PASTA_ARQUIVOS)
        options = webdriver.ChromeOptions()
        options.add_experimental_option("prefs", {
            "download.default_directory": path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })
        options.add_argument("--headless")

        servico = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=servico,options=options)
    except Exception as e:
         logger.error(f'Erro na inicialização do selenium : {e}')
         
    driver.get(r'https://www2.susep.gov.br/menuestatistica/ses/principal.aspx')

    time.sleep(5)

    id = driver.find_element(By.ID,'GEEST')
    if id: 
        try:
            tag_a = id.find_element(By.TAG_NAME,'a')
            link = tag_a.get_attribute('href')
        except Exception as e:
            logger.error(f'Obtendo link do download {e}')
    
        if link:
            driver.get(link)
            logger.info('Efetuando download do Zip')
        
        else: 
            logger.error('Link não encontrado')
            print('link não encontrado')
            driver.close()
            return

    else: 
        logger.error('Id do botão não localizado')
        print('Id do botão não localizado')
        driver.close()
        return
        
    time.sleep(5)
    segundos = 0
    
    while True:
        if segundos % 10 == 0:
                print('Efetuando Download')
        
        segundos += 1
        if segundos > qde_max_segundos: 
            logger.error('Excedeu o tempo limite de download')
            print('Excedeu o tempo limite de download')
            driver.close()
            return check_download
        
        arquivo = glob(os.path.join(path,'*.crdownload'))
        if arquivo:
            time.sleep(1)
            continue
        break
    
    print('Download Concluído')
    logger.info('Download Concluído')
    driver.close()
    check_download = True
    return check_download

In [5]:

check_download = buscar_arquivo_na_susep(path=PATH_PASTA_ARQUIVOS,
                        qde_max_segundos_download=QDE_MAX_SEGUNDOS_DOWNLOAD)

2025-03-02 01:56:49.310 | INFO     | __main__:buscar_arquivo_na_susep:12 - Efetuando download do Zip
2025-03-02 01:56:59.684 | INFO     | __main__:buscar_arquivo_na_susep:39 - Efetuando download do Zip


Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download


2025-03-02 01:58:08.757 | INFO     | __main__:buscar_arquivo_na_susep:66 - Download Concluído


Download Concluído


In [6]:
def descompactar_zip(path_origem,check_sucesso:bool = False) -> bool:
    '''
    Objetivo da função: Verifica se existe arquivos .zip e efetua a descompactação
    Args:
        path_origem (Path): Caminho onde iremos procurar os arquivos .zip
        check_sucesso (int) : Essa váriavel é o retorno da função  "buscar_arquivo_na_susep()" , se tudo correu dentro do esperado ela terá o valor True e prosseguiremos com a descompactação 

    Returns:
        bool: Se a descompactação for concluída com succeso retornará True caso contrário False 
        
    '''
    
    try:
        if check_sucesso:
            arquivos_csv =  glob(os.path.join(path_origem,'*.csv'))
            for arq in arquivos_csv:
                os.remove(arq)

        time.sleep(5)
        if check_sucesso:
            arquivos = glob(os.path.join(path_origem,'*.zip'))

            for arquivo in arquivos:
                if arquivo.endswith('.zip'):
                    caminho_completo = os.path.join(path_origem, arquivo)

                    with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
                        zip_ref.extractall(path_origem)
        check_zip =True
        logger.info('Descompactação efetuada')
    except Exception as e:
        check_zip = False
        logger.error(f'Erro na Descompactação {e}')
    return check_zip

In [7]:
check_zip = descompactar_zip(path_origem=PATH_PASTA_ARQUIVOS,check_sucesso=check_download)

2025-03-02 01:58:36.863 | INFO     | __main__:descompactar_zip:30 - Descompactação efetuada


In [8]:
import importlib
importlib.reload(schemas)

<module 'schemas' from 'c:\\PROJETOS_PYTHON\\WS_SES_SUSEP\\src\\schemas.py'>

In [9]:
def validar_schemas_obrigatorios(path:Path,check_sucesso_zip:bool = False):
    '''  
    Objetivo da função: Efetua a validação das estrutura dos arquivos csv
    Args:
        path_origem (Path): Caminho onde iremos procurar os arquivos .csv
        check_sucesso_zip (int) : Essa váriavel é o retorno da função  "descompactar_zip()" , se tudo correu dentro do esperado ela terá o valor True e prosseguiremos com a validação 

    Returns:
        bool: Se a validação for concluída com succeso retornará True caso contrário False
    '''

    check_validacao = True
    encoding_utilizado = 'ISO-8859-1'
    separador = ';'
    decimal = ','
    lazy = True

    if check_sucesso_zip:

        try:
            print('Ses_cias')
            schemas.SchemaSesCias.validate(pd.read_csv(os.path.join(path,"Ses_cias.csv"),encoding=encoding_utilizado,sep=separador),lazy=  lazy)
            print('Ses_ramos')
            schemas.SchemaSesRamos.validate(pd.read_csv(os.path.join(path,"Ses_ramos.csv"),encoding=encoding_utilizado,sep=separador),lazy=  lazy)
            print('ses_gruposramos')
            schemas.SchemaSesGrupoRamo(pd.read_csv(os.path.join(path,"ses_gruposramos.csv"),encoding=encoding_utilizado,sep=separador),lazy=  lazy)
            print('Ses_seguros')
            schemas.SchemaSesSeguros.validate(pd.read_csv(os.path.join(path,"Ses_seguros.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('SES_UF2')
            schemas.SesUf2.validate(pd.read_csv(os.path.join(path,"SES_UF2.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('Ses_campos')
            schemas.SchemaSesCampos.validate(pd.read_csv(os.path.join(path,"Ses_campos.csv"),encoding=encoding_utilizado,sep=separador),lazy=  lazy)
            print('SES_Balanco')
            schemas.SchemaSesBalanco.validate(pd.read_csv(os.path.join(path,"SES_Balanco.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('Ses_Dados_Cap')
            schemas.SchemaSesDadosCap.validate(pd.read_csv(os.path.join(path,"Ses_Dados_Cap.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('ses_cap_uf')
            schemas.SchemaSesCapUf.validate(pd.read_csv(os.path.join(path,"ses_cap_uf.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('SES_ValoresMovRamos')
            schemas.SchemaSesValoresMovRamos.validate(pd.read_csv(os.path.join(path,"SES_ValoresMovRamos.csv"),encoding=encoding_utilizado,sep=separador,decimal=decimal),lazy=  lazy)
            print('ses_valoresresmovgrupos')
            schemas.SchemaSesValoresResMovMovGrupos.validate(pd.read_csv(os.path.join(path,"ses_valoresresmovgrupos.csv"),encoding = encoding_utilizado, sep = separador , decimal=decimal ) ,lazy =  lazy)

            print('Sucesso na validação')
            logger.info(f'Schemas validados com sucesso')
            return check_validacao
        except Exception as e:
            check_validacao = False
            logger.error(f'Erro na validação dos Schemas {e}')
            return  check_validacao

In [10]:
check_validacao = validar_schemas_obrigatorios(PATH_PASTA_ARQUIVOS,check_zip)

Ses_cias
Ses_ramos
ses_gruposramos
Ses_seguros
SES_UF2
Ses_campos
SES_Balanco
Ses_Dados_Cap
ses_cap_uf
SES_ValoresMovRamos
ses_valoresresmovgrupos


2025-03-02 02:00:00.598 | INFO     | __main__:validar_schemas_obrigatorios:45 - Schemas validados com sucesso


Sucesso na validação


In [11]:
def mover_arquivos(path_origem,path_destino,check_validacao):
    if check_validacao:
        try:
            criar_pasta(path_destino)
            arquivos =  glob(os.path.join(path_origem,'*.csv'))
            for arq in arquivos:
                shutil.copy(arq,path_destino)
            logger.info('Arquivos copiados com sucesso')
            print('processo finalizado')
        except Exception as e :
            logger.error(f'Erro ao copiar os arquivos {e}')
    

In [12]:
mover_arquivos(PATH_PASTA_ARQUIVOS,PATH_PASTA_ARQUIVOS_CSV,check_validacao)

2025-03-02 02:00:03.922 | INFO     | __main__:mover_arquivos:8 - Arquivos copiados com sucesso


processo finalizado


In [ ]:
try:
    check_download = False
    qde_max_segundos = 1000
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": r"C:\Users\thiag\Downloads",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    #options.add_argument("--headless")

    servico = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=servico,options=options)
except Exception as e:
        print(e)
        #logger.error(f'Erro na inicialização do selenium : {e}')
        ...
driver.get(r'https://www2.susep.gov.br/menuestatistica/ses/principal.aspx')

time.sleep(5)

id = driver.find_element(By.ID,'GEEST')
tag_a = id.find_element(By.TAG_NAME,'a')
link = tag_a.get_attribute('href')

In [ ]:
link = tag_a.get_attribute('href')

In [13]:
import pandas as pd